In [75]:
### Estratégia Dias em Queda (4)

In [76]:

# Imports

import yfinance as yf
import investpy as inv
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [77]:
# Definindo a lista de ativos que compõem o índice IBXX a partir de arquivo excel
arquivo = pd.read_excel("ibxx.xlsx")
ibxx = arquivo['Código'].sort_values().tolist() #ordem alfabética, transforma em lista
for i in range(len(ibxx)): #Adiciona '.SA' a cada ativo, para trabalhar com yfinance
    ibxx[i] = ibxx[i]+".SA"

In [78]:
# Carrega os dados de fechamentos de todos os ativos que compõesm o índice
start = '2000-01-01'
end = datetime.today() #Hoje#

#cria um dataframe com os preços de fechamento dos ativos da lista:

df = yf.download(tickers=ibxx, start=start, end=end).copy()["Adj Close"].round(decimals=2)
# Inicializa um dicionário vazio
data_dict = {}
#itera através das colunas do dataframe
for nome_coluna, serie in df.iteritems():
    #cria um novo dataframe a partir da coluna
    novo_df = pd.DataFrame({nome_coluna: serie})
    #adiciona o novo df ao dict com o nome da coluna como chave
    data_dict[nome_coluna] = novo_df


[*********************100%***********************]  99 of 99 completed


In [79]:
quatro_dias_dict = {} # Inicializa um dicionário para armazenar as listas de chaves
for key, df in data_dict.items():
    quatro_dias = []
    # Localizar quatro dias em queda:
    for coluna in df.columns:
        #Seleciona as últimas 4 linhas da coluna
        ultimas_6_linhas = df[coluna].tail(6)
        #Verifica se as últimas 4 linhas estão em ordem decrescente e se a quinta é maior que a sexta
        cond1 = (ultimas_6_linhas.iloc[-1] < ultimas_6_linhas.iloc[-2]).all() and \
            (ultimas_6_linhas.iloc[-2] < ultimas_6_linhas.iloc[-3]).all() and \
            (ultimas_6_linhas.iloc[-3] < ultimas_6_linhas.iloc[-4]).all() and \
            (ultimas_6_linhas.iloc[-4] < ultimas_6_linhas.iloc[-5]).all()
        cond2 = (ultimas_6_linhas.iloc[-5] > ultimas_6_linhas.iloc[-6]).all()

        quatro_em_queda = cond1 and cond2
        if quatro_em_queda:
            quatro_dias.append(key) # Adiciona a chave do dataframe
    quatro_dias_dict[key] = quatro_dias # Armazena a lista no dicionário

quatro_dias = [] # Inicializa a lista para armazenar as chaves
for key in quatro_dias_dict:
    quatro_dias += quatro_dias_dict[key] # Concatena as listas de chaves


In [80]:
#Exibe quais ativos se enquadram na condição:
if not quatro_dias:
    print("Nenhum ativo se enquadra na condição em {}".format(df.index[-1].strftime('%Y-%m-%d')))
else:
    print("Ativos em queda por 4 dias consecutivos em {}:".format(df.index[-1].strftime('%Y-%m-%d')))
    print(quatro_dias)

Ativos em queda por 4 dias consecutivos em 2023-04-05:
['ASAI3.SA', 'AZUL4.SA', 'CBAV3.SA', 'CRFB3.SA', 'CSAN3.SA', 'CSNA3.SA', 'CVCB3.SA', 'GOLL4.SA', 'LREN3.SA', 'PCAR3.SA', 'USIM5.SA', 'VAMO3.SA']


In [81]:
# Cria um dicionário com dataframes com ativo na lista "quatro_dias"
dict_enquadrados = {}
#Loop para criar um dataframe para cada nome na lista
for ativo in quatro_dias:
    #cria um df vazio com o nome
    dfx = pd.DataFrame()
    dfx.name = ativo
    # Adiciona o DataFrame ao dicionário, usando o nome como chave
    dict_enquadrados[ativo] = dfx
dict_enquadrados = {k: v for k, v in data_dict.items() if k in quatro_dias}

#legal, a parir daqui eu travei na parte de automação para montar cada backtest dentro de cada df dentro do dicionário dict_enquadrados
#vou prosseguir montando manualmente'''


In [119]:
# Adiciona coluna 'sinal' ao dataframe selecionado e calcula os retorno para os diferentes prazos:
#Seleciona um dataframe
ativo_bt = 'AZUL4.SA'
df_bt = dict_enquadrados['{}'.format(ativo_bt)]
# Cria uma coluna 'sinal' inicialmente com valores nulos
df_bt['sinal'] = np.nan
# Percorre todas as linhas do dataframe
for i in range(5, len(df)):
    # Seleciona as últimas 6 linhas da coluna
    ultimas_6_linhas = df_bt.iloc[i-5:i+1, 0]
    
    # Verifica se as últimas 4 linhas estão em ordem decrescente e se a quinta é maior que a sexta
    cond1 = (ultimas_6_linhas.iloc[-1] < ultimas_6_linhas.iloc[-2]).all() and \
            (ultimas_6_linhas.iloc[-2] < ultimas_6_linhas.iloc[-3]).all() and \
            (ultimas_6_linhas.iloc[-3] < ultimas_6_linhas.iloc[-4]).all() and \
            (ultimas_6_linhas.iloc[-4] < ultimas_6_linhas.iloc[-5]).all()
    cond2 = (ultimas_6_linhas.iloc[-5] > ultimas_6_linhas.iloc[-6]).all()
    
    quatro_em_queda = cond1 and cond2
    
    # Atribui 1 para a coluna 'sinal' se a condição for válida, 0 se não
    if quatro_em_queda:
        df_bt.loc[df.index[i], 'sinal'] = 1
    else:
        df_bt.loc[df.index[i], 'sinal'] = 0
    
for i in range (1,11): #TEM ERRO AQUI, NÃO ESTÁ TRAZENDO OS VALORES CORRETOS
    df_bt[f'{ativo_bt}_{i}'] = df_bt[ativo_bt].shift(i)
    #Retornos % nos diferentes targets:
#for i in range(1, 11):
#    col_name = f'{ativo_bt}_{i}'
#    df_bt[col_name] = ((df_bt[col_name] / df_bt[ativo_bt])-1)
#drop nan e valor de sinal = 0:
df_bt = df_bt.dropna(axis=0)
df_bt = df_bt.drop(df_bt[df_bt['sinal'] == 0].index)



# Criando o novo dataframe com as informações desejadas
total_trades = len(df_bt)
retorno_total = df_bt.iloc[:, 2:].sum()
percentual_positivos = (df_bt.iloc[:, 2:] > 0).mean() * 100
percentual_negativos = (df_bt.iloc[:, 2:] < 0).mean() * 100
media_positivos = df_bt.iloc[:, 2:][df_bt.iloc[:, 2:] > 0].mean()
media_negativos = df_bt.iloc[:, 2:][df_bt.iloc[:, 2:] < 0].mean()

novo_df = pd.DataFrame({
    'total de trades': total_trades,
    'retorno total': retorno_total,
    '% positivos': percentual_positivos,
    '% negativos': percentual_negativos,
    'média positivos': media_positivos,
    'média negativos': media_negativos
})

In [121]:
# Adiciona os targets de 1 a 10 dias
print(df_bt)
novo_df
df_preço = dict_enquadrados['AZUL4.SA']

            AZUL4.SA  sinal  AZUL4.SA_1  AZUL4.SA_2  AZUL4.SA_3  AZUL4.SA_4  \
Date                                                                          
2017-05-11     25.25    1.0       25.69       25.80       26.00       26.60   
2017-09-27     28.77    1.0       28.88       29.40       29.85       30.39   
2017-12-08     26.90    1.0       26.99       27.00       27.07       27.25   
2018-01-10     26.45    1.0       26.80       26.91       27.20       27.23   
2018-03-02     32.90    1.0       33.33       33.63       33.67       33.76   
2018-03-19     36.38    1.0       36.41       36.82       37.00       37.18   
2018-05-07     33.00    1.0       33.48       33.78       35.51       36.40   
2018-05-16     27.30    1.0       27.90       29.42       30.56       32.00   
2018-06-08     23.20    1.0       23.51       24.42       25.40       27.00   
2018-07-31     24.13    1.0       24.32       25.29       25.47       25.76   
2018-08-24     22.00    1.0       22.05       22.45 